In [5]:
import os
import warnings
import glob
import numpy as np
import pandas as pd
from skimage import io
import gaussianfunctions_KM as gf
from skimage.measure import label, regionprops,regionprops_table
from skimage import data, util, measure
from scipy import ndimage
import numpy.ma as ma
from matplotlib.patches import Ellipse
from skimage.draw import ellipse
from skimage import draw
from scipy.ndimage import gaussian_filter
warnings.filterwarnings("ignore")
io.use_plugin('matplotlib')

def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

def get_subdirs(dir):
    return next(os.walk(dir))[1]


def gaussian(x, amplitude, mean, stddev, offset):
    return (amplitude * np.exp(-((x - mean) / 4 / stddev)**2)+offset)    


       

directory = "path"
conditions = get_subdirs(directory)

fov_list = []
track_id_list = []
for condition in conditions:

    os.chdir(directory +"\\"+condition)

    #get image name
    image_name = ''.join([f for f in glob.glob("*ome.tif")])
    
    #get trackmate result file name
    result_file_name = glob.glob("*.csv")
    
    
    if len(result_file_name)>0:
        for s in result_file_name:
            #read in movie
            movie = io.imread(image_name).astype(np.float64)
            
            #get YAP channel
            YAP_channel = movie[:,:,:,2]  

            #read in trackmate result file
            trackmate_result = pd.read_csv(s)

            #remove the first two rows that are duplicates of the header (default trackmate output format)
            tm = trackmate_result.drop([0, 1,2])

            #change frames to float
            tm["FRAME"]= tm["FRAME"].astype(float)
            
            #make a copy of the trackmate df to add spot quantifications
            tm_copy = tm
            
            #add new columns to append results
            tm_copy["new_frame"]="" 
            tm_copy['YAP_integral_int']=""
            tm_copy['YAP_dilute_phase_mean']=""

            #extract all IDs
            track_list = tm["ID"].to_list()
            
            #convert all IDs to integers 
            spot_list = [float(i) for i in track_list]

            #iterate through every track
            for i in spot_list:
                
                #convert track IDs to floats              
                tm["ID"]= tm["ID"].astype(float)
            
                #select track
                current = tm.loc[tm["ID"] == i]

                #extract all frames of the track
                frame_list = current['FRAME'].to_list()  
                int_frame_list = [int(i) for i in frame_list]

                #iterate through frames
                for d in int_frame_list:
                                   
                                    f = int(d)
                                    
                                    #get x,y coordinate of the spot
                                    current_df = current[current['FRAME']==float(f)]
                                    x_pos = current_df["POSITION_Y"].values[0]
                                    y_pos = current_df["POSITION_X"].values[0]
                                    
                                    #get the current frame of the movie
                                    current_img = YAP_channel[f,:,:]
                                    
                                    #select a small region around the spot on the image
                                    y_min = round(float(y_pos)) - 7
                                    y_max = round(float(y_pos)) + 8
                                    x_min = round(float(x_pos)) - 7
                                    x_max = round(float(x_pos)) + 8

                                    local_img = current_img[x_min:x_max, y_min:y_max]

                                    #-----------Extimate fitting parameters and fit guassian to trackmate-identified YAP spots-------------
                                    #get local background from a slightly larger area 
                                    y_min_global = np.int64(float(y_pos) - 10)
                                    y_max_global = np.int64(float(y_pos) + 11)
                                    x_min_global = np.int64(float(x_pos) - 10)
                                    x_max_global = np.int64(float(x_pos) + 11)
                                    local_img_global = current_img[x_min_global:x_max_global, y_min_global:y_max_global]

                                    thresh = np.median(local_img_global)*0.5

                                    #For spots that are at the edge of a nucleus, the gaussian fit open recognizes the nucleus edge instead of the spot.
                                    #To avoid this, fill in pixel outside of the nucleus with the average nuclear background value 
                                    local_img_masked = ma.masked_where(local_img <= thresh, local_img)
                                    local_img_masked_filled = local_img_masked.filled(fill_value=np.mean(local_img_masked))

                                    #estimate offset
                                    off_guess_img = np.median(local_img_global[local_img_global>100]) 
                                    local_median_img = off_guess_img

                                    #compute min gaussheight to be at least 10% above offset
                                    height_low_bound_img = (1.1 * off_guess_img) - off_guess_img; 
                                    
                                    #get local maximum to estimate amplitude
                                    local_max_value_img = np.max(local_img_masked)
                                    amp_guess_img = local_max_value_img - off_guess_img                           

                                    # inital guesses for gaussian fit
                                    initial_guess_img = (amp_guess_img, 7, 7, 1.0, 1.0, off_guess_img)

                                    #set boundary conditions for gaussian fitting
                                    bound_tup_img = ([height_low_bound_img, 7, 7, 1, 1, 0], [+np.Inf, 8, 8, 2.5, 2.5, +np.Inf])

                                    #Generate grid for gaussian fitting
                                    x_local_gauss_img = np.linspace(0, 15 - 1, 15)
                                    y_local_gauss_img = np.linspace(0, 15 - 1, 15)
                                    x_local_gauss_img,y_local_gauss_img = np.meshgrid(x_local_gauss_img, y_local_gauss_img)

                                    #Fit 2D gaussain to local YAP image
                                    param_img = gf.fit(gf.func, (x_local_gauss_img,y_local_gauss_img), local_img_masked_filled, initial_guess_img, bound_tup_img) ##Fit gaussian


                    
                                    #-------------quantify YAP hub integral intensity and dilute phase intensity----------------
                                    #subtract background from YAP image
                                    local_img_global_bkgd_subtr = local_img - param_img.offset

                                    #if background subtraction results in negative values, replace them with zeros
                                    local_img_global_bkgd_subtr_masked = ma.masked_where(local_img_global_bkgd_subtr <= 0, local_img_global_bkgd_subtr)
                                    local_img_global_bkgd_subtr_masked_filled = local_img_global_bkgd_subtr_masked.filled(fill_value=0)

                                    # generate an ellipse array based on gaussian fit parameters (will be used as image mask)
                                    ellipse_outline = Ellipse((param_img.center_x, param_img.center_y), width=param_img.sigma_x*3, height=param_img.sigma_y*3,linewidth=1, edgecolor='r', facecolor='none')
                                    ellipse_array = np.zeros((local_img_global_bkgd_subtr_masked_filled.shape[0], local_img_global_bkgd_subtr_masked_filled.shape[1]))
                                    rr, cc = ellipse(param_img.center_y, param_img.center_x, param_img.sigma_y*3/2, param_img.sigma_x*3/2)
                                    ellipse_array[rr,cc]=1
                                    ellipse_label = label(ellipse_array)
                                    
                                    #mask local YAP image outside of gaussian fit
                                    local_img_global_bkgd_subtr_masked_filled_spot = ma.masked_where(ellipse_label == 0, local_img_global_bkgd_subtr_masked_filled)
                                    
                                    #fill masked values with zeros and quantify sum intensity of YAP hub pixel (=dense phase YAP intensity)
                                    local_img_global_bkgd_subtr_masked_filled_spot_quant = local_img_global_bkgd_subtr_masked_filled_spot.filled(fill_value=0)
                                    dense_phase_intensity = np.sum(local_img_global_bkgd_subtr_masked_filled_spot_quant)
                
                                    #Quantify dilute phase intensity in vicinity to the YAP hub
                                    local_img_global_bkgd_subtr_masked_filled_dilute_quant = ma.masked_where(ellipse_label == 1, local_img_global_bkgd_subtr_masked_filled)
                                    dilute_phase_intensity = np.mean(local_img_global_bkgd_subtr_masked_filled_dilute_quant) 

                                    #------------enter results into dataframe-----------
                                    #select row in dataframe
                                    matched_row = (tm_copy['FRAME'] == float(f))&(tm_copy['ID']==float(i))

                                    #enter quantifications into existing dataframe
                                    tm_copy.loc[matched_row,'new_frame'] = str(f)

                                    #if image contained no YAP spots, enter zeros, else enter results
                                    if param_img.amp == 0:
                                        tm_copy.loc[matched_row,'YAP_integral_int'] = str([0])
                                        tm_copy.loc[matched_row,'YAP_dilute_phase_mean'] = str([0])
                                        
                                    else:
                                        tm_copy.loc[matched_row,'YAP_integral_int'] = str(dense_phase_intensity)
                                        tm_copy.loc[matched_row,'YAP_dilute_phase_mean'] = str(dilute_phase_intensity)


            tm_copy.to_csv(directory + "\\"+s+"_result.csv")  
  

<tifffile.TiffFile 'fov_3_MMStack_Default.ome.tif'> MMStack series is missing files. Returning subset (500, 1, 3, 1) of (500, 1, 3, 1)
<tifffile.TiffFile 'fov_3_MMStack_Default.ome.tif'> MMStack is missing 498 pages. Missing data are zeroed
<tifffile.TiffFile 'fov_3_MMStack_Default.ome.tif'> MMStack series is missing files. Returning subset (500, 1, 3, 1) of (500, 1, 3, 1)
<tifffile.TiffFile 'fov_3_MMStack_Default.ome.tif'> MMStack is missing 498 pages. Missing data are zeroed
<tifffile.TiffFile 'fov_3_MMStack_Default.ome.tif'> MMStack series is missing files. Returning subset (500, 1, 3, 1) of (500, 1, 3, 1)
<tifffile.TiffFile 'fov_3_MMStack_Default.ome.tif'> MMStack is missing 498 pages. Missing data are zeroed
<tifffile.TiffFile 'fov_1_MMStack_Default.ome.tif'> MMStack series is missing files. Returning subset (500, 1, 3, 1) of (500, 1, 3, 1)
<tifffile.TiffFile 'fov_1_MMStack_Default.ome.tif'> MMStack is missing 498 pages. Missing data are zeroed
<tifffile.TiffFile 'fov_1_MMStack_De

ValueError
ValueError
ValueError


<tifffile.TiffFile 'fov_2_MMStack_Default.ome.tif'> MMStack series is missing files. Returning subset (500, 1, 3, 1) of (500, 1, 3, 1)
<tifffile.TiffFile 'fov_2_MMStack_Default.ome.tif'> MMStack is missing 498 pages. Missing data are zeroed
<tifffile.TiffFile 'fov_2_MMStack_Default.ome.tif'> MMStack series is missing files. Returning subset (500, 1, 3, 1) of (500, 1, 3, 1)
<tifffile.TiffFile 'fov_2_MMStack_Default.ome.tif'> MMStack is missing 498 pages. Missing data are zeroed


In [ ]:
import os

os.chdir('C:\\Users\\kirstin\\Desktop\\python')
import scipy.optimize as opt
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import argparse
import pickle
import time
import math
from skimage import io
io.use_plugin('matplotlib')
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
#import pyMods as pyM  
import gaussianfunctions_KM as gf
from scipy.interpolate import griddata
from scipy import stats
from pandas.core.window import Rolling
from skimage.measure import label, regionprops,regionprops_table
from skimage.color import label2rgb
from skimage.morphology import erosion, dilation, opening, closing
from skimage.morphology import disk

from skimage import data, util, measure
from skimage.segmentation import clear_border

#from skimage.external.tifffile import imsave
import peakutils
from PIL import Image
import tifffile
from scipy import ndimage
from skimage import img_as_ubyte
import shapely
from shapely.geometry import Polygon, Point
import statsmodels.api as sm
import numpy.ma as ma
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from matplotlib.patches import Ellipse
from skimage import img_as_uint
from skimage.draw import ellipse
from shapely.ops import nearest_points
from skimage import draw
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Circle
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

#generate output dataframe (one for the raw values of the med1 intenisty over time, and one for the average nuclear properties)

mean_out_df = pd.DataFrame()


YAP = []
#define directory 
directory = "Z:\\Weiner_Lab\\kirstinmeyer\\scope\\SINGLE-MOLECULE_IMAGING\\KY612_240403_Med_YAP_hubs_with_charge\\GFP(+30)-Halo-YAP\\"

#read in list of all result files in the current directory
os.chdir(directory)
csv_files = glob.glob("*result.csv")

#iterate through individual result files
for csv in csv_files:
    
    #extract the roi from the csv file name
    roi = csv.split("_")[-3]
    roi_number = int(''.join(filter(str.isdigit, roi)))
    
    
    #read in 
    current_file = pd.read_csv(directory +csv)
    
    
    #read in file
    nuc_YAP_file = pd.read_csv(directory +csv.partition("_SPOTS")[0]+"_nuc_intensity.csv")
    
    #select the dataframe row that corresponds to the current nucleus roi (row 0 = roi 0, row 1 = roi 1 etc.)
    current_roi = nuc_YAP_file.iloc[roi_number]

    #get mean nuclear YAP intensity and area of the nucleus
    nuc_YAP = current_roi['Mean']
    #nuc_area = current_roi['Area']
    
    print(nuc_YAP)
    if (int(nuc_YAP)>=150)&(int(nuc_YAP)<=852):
        curr_YAP = []
        curr_YAP = current_file['YAP_amplitude'].to_list()
        YAP = YAP + curr_YAP

mean_out_df["YAP_amplitude"] = pd.Series(YAP)


        
mean_out_df.to_csv(directory +"YAP_single_spot_amplitudes_max_endo_YAP_levels_only.csv")    


In [1]:
import os

os.chdir('C:\\Users\\kirstin\\Desktop\\python')
import scipy.optimize as opt
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import argparse
import pickle
import time
import math
from skimage import io
io.use_plugin('matplotlib')
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
#import pyMods as pyM  
import gaussianfunctions_KM as gf
from scipy.interpolate import griddata
from scipy import stats
from pandas.core.window import Rolling
from skimage.measure import label, regionprops,regionprops_table
from skimage.color import label2rgb
from skimage.morphology import erosion, dilation, opening, closing
from skimage.morphology import disk

from skimage import data, util, measure
from skimage.segmentation import clear_border

#from skimage.external.tifffile import imsave
import peakutils
from PIL import Image
import tifffile
from scipy import ndimage
from skimage import img_as_ubyte
import shapely
from shapely.geometry import Polygon, Point
import statsmodels.api as sm
import numpy.ma as ma
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from matplotlib.patches import Ellipse
from skimage import img_as_uint
from skimage.draw import ellipse
from shapely.ops import nearest_points
from skimage import draw
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Circle
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

#generate output dataframe (one for the raw values of the med1 intenisty over time, and one for the average nuclear properties)
single_nuc_out = pd.DataFrame()
mean_out_df = pd.DataFrame()
single_mol_amp = 212
per_nuc_average = []
YAP = []

directory = "Z:\\Weiner_Lab\\kirstinmeyer\\YAP_condensate_paper\\YAP_hubs_vs_MED_NEW\\KY658_240630_YAP-SPT_Hubs\\SNAP-MED_Halo-YAP\\"

#read in list of all result files in the current directory
os.chdir(directory)
csv_files = glob.glob("*result.csv")


for csv in csv_files:
    duration_per_nuc = []
    all_per_nuc = []
    all_max = []
    
    #extract the roi from the csv file name
    roi = csv.split("_")[-3]
    roi_number = int(''.join(filter(str.isdigit, roi)))
    
    #read in 
    current_file = pd.read_csv(directory +csv)
    #current_file = raw_df.drop([0, 1,2])
    
    #read in file
    nuc_YAP_file = pd.read_csv(directory +csv.partition("_SPOTS")[0]+"_nuc_intensity.csv")
    
    #select the dataframe row that corresponds to the current nucleus roi (row 0 = roi 0, row 1 = roi 1 etc.)
    current_roi = nuc_YAP_file.iloc[roi_number]

    #get mean nuclear YAP intensity and area of the nucleus
    nuc_YAP = current_roi['Mean']   
    nuc_area = current_roi['Area']
    
    tracks = current_file['TRACK_ID']
    multi_frame_tracks = tracks.dropna(how='all')
    
    
    track_ID_int = [int(x) for x in multi_frame_tracks]
    unique_ids = unique(track_ID_int)
    print(unique_ids)
    
    for i in unique_ids:
        
            current_file["TRACK_ID"]= current_file["TRACK_ID"].astype(float)
            data = current_file.loc[current_file['TRACK_ID'] == float(i)]
            data4 = data.sort_values(by='new_frame',ascending=True, key=pd.to_numeric)
            
            #for trackmate, I allow a gap of two frames for calling it the same track. These are not listed in the original trackmate file.
            #I am therefore closing potential gaps for the time frames
            
            timeseries = data4['new_frame'].to_list()
            time_min = np.min(timeseries)
            time_max = np.max(timeseries)
            frame_list_full = [*range(time_min, time_max+1, 1)]       
            
            #making sure I am not copying over data from the previous loop, i am setting the list to empty
            YAP_timeseries = []
            YAP_timeseries = data4['YAP_integral_int'].to_list()
            for index, item in enumerate(YAP_timeseries):
                if item == "[0]":
                    YAP_timeseries[index] = "0"
            
            YAP_timeseries_float = [float(x) for x in YAP_timeseries]
            
            #only consider trace a"YAP hub" if intenisty at any point is >=5 molecules
            if any(float(num) >= 5*single_mol_amp  for num in YAP_timeseries_float): 
                YAP_min = np.min(YAP_timeseries_float)
                YAP_max = np.max(YAP_timeseries_float)
                YAP_mean = np.mean(YAP_timeseries_float)
                YAP_median = np.median(YAP_timeseries_float)
                duration = len(frame_list_full)
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [YAP_min] + [YAP_max] + [YAP_mean] + [YAP_median] + [duration]
                mean_out_df = mean_out_df.append(pd.Series(out),ignore_index=True)           
                
                all_per_nuc = all_per_nuc + [YAP_mean]
                duration_per_nuc = duration_per_nuc + [duration]
                all_max = all_max + [YAP_max]
                
            else:
                
                YAP_min = np.nan
                YAP_max = np.nan
                YAP_mean = np.nan
                YAP_median = np.nan

                duration = np.nan
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [YAP_min] + [YAP_max] + [YAP_mean] + [YAP_median] + [duration]
                mean_out_df =mean_out_df.append(pd.Series(out),ignore_index=True)
                #mean_out_df[str(csv) + str(i)] = pd.Series(out)
    mean_YAP_per_nuc = np.sum(all_per_nuc)
    median_YAP_per_nuc = np.median(all_per_nuc)
    num_hubs_per_nuc = len(all_per_nuc)
    mean_duration = np.mean(duration_per_nuc)
    mean_YAP_max = np.sum(all_max)
    
    per_nuc_out = [csv]+ [nuc_YAP]+ [nuc_area]+ [mean_YAP_per_nuc] + [mean_YAP_max]
    single_nuc_out =single_nuc_out.append(pd.Series(per_nuc_out),ignore_index=True)
    
single_nuc_out.to_csv(directory +"KY658_Halo-YAP_hub_features_if more than5molecules.csv")           
#mean_out_df.to_csv(directory +"YAP_hub_features.csv")    


[3, 2, 5, 10, 12, 11, 6, 7, 0, 1, 13, 9, 4, 8, 18, 20, 14, 19, 17, 15, 16, 22, 23, 21, 24, 25, 26, 27, 28, 61, 59, 60, 35, 29, 76, 34, 63, 32, 36, 33, 30, 31, 40, 37, 38, 39, 41, 43, 44, 42, 45, 46, 47, 48, 160, 51, 50, 49, 52, 56, 58, 110, 153, 143, 57, 55, 53, 54, 128, 119, 120, 106, 107, 62, 64, 65, 66, 67, 68, 69, 74, 71, 75, 70, 72, 73, 77, 78, 79, 103, 82, 83, 84, 80, 81, 85, 86, 87, 88, 89, 90, 91, 93, 92, 94, 95, 96, 97, 98, 100, 101, 99, 102, 215, 104, 105, 108, 109, 209, 111, 112, 113, 114, 117, 115, 118, 116, 122, 121, 123, 125, 124, 126, 127, 129, 130, 131, 132, 135, 134, 206, 137, 205, 136, 133, 138, 139, 140, 141, 142, 197, 198, 144, 145, 146, 147, 148, 150, 149, 152, 151, 183, 156, 154, 155, 158, 159, 157, 161, 162, 163, 165, 275, 164, 167, 166, 168, 270, 169, 171, 170, 173, 172, 174, 175, 176, 177, 178, 179, 180, 181, 182, 269, 263, 262, 184, 185, 186, 188, 187, 191, 190, 189, 192, 193, 194, 195, 196, 200, 199, 201, 202, 203, 204, 207, 208, 210, 243, 214, 236, 211, 230,

[13, 17, 15, 22, 7, 18, 11, 19, 20, 21, 16, 23, 0, 24, 25, 3, 26, 4, 27, 6, 8, 28, 29, 12, 1, 2, 5, 9, 10, 14, 51, 59, 58, 56, 57, 48, 43, 38, 50, 49, 30, 52, 40, 33, 31, 35, 53, 54, 34, 55, 36, 32, 37, 39, 41, 42, 44, 45, 46, 47, 60, 61, 62, 64, 66, 68, 63, 65, 67, 72, 73, 69, 70, 71, 75, 77, 74, 76, 78, 79, 80, 81, 82, 117, 226, 114, 113, 111, 112, 101, 86, 110, 88, 89, 91, 83, 116, 115, 84, 90, 93, 85, 87, 92, 99, 102, 98, 100, 94, 95, 96, 97, 107, 157, 105, 103, 104, 106, 108, 109, 118, 119, 120, 121, 122, 123, 124, 244, 128, 125, 245, 130, 126, 127, 218, 217, 201, 219, 136, 145, 143, 133, 200, 202, 203, 142, 186, 187, 188, 189, 151, 185, 134, 131, 148, 138, 149, 152, 146, 147, 135, 132, 150, 129, 139, 141, 144, 137, 140, 190, 167, 191, 154, 155, 156, 153, 164, 176, 177, 179, 180, 166, 178, 158, 159, 160, 163, 165, 161, 162, 171, 174, 172, 168, 175, 169, 170, 173, 199, 364, 380, 181, 183, 184, 182, 345, 193, 346, 197, 347, 195, 196, 192, 194, 198, 333, 213, 216, 205, 204, 210, 211,

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 14, 15, 17, 18, 19, 20, 21, 30, 38, 39, 40, 33, 41, 47, 43, 46, 45, 48, 49, 42, 50, 32, 44, 27, 36, 31, 28, 34, 35, 29, 37, 22, 23, 25, 26, 24, 52, 51, 54, 55, 56, 57, 53, 60, 61, 58, 59, 63, 62, 66, 64, 65, 67, 68, 72, 69, 70, 71, 73, 76, 74, 75, 77, 282, 118, 86, 280, 281, 283, 82, 92, 89, 236, 237, 91, 88, 90, 117, 114, 78, 80, 115, 116, 83, 87, 84, 85, 197, 81, 79, 124, 98, 122, 180, 123, 102, 109, 125, 113, 120, 121, 110, 111, 112, 99, 101, 96, 93, 100, 94, 95, 103, 105, 119, 97, 104, 106, 107, 108, 135, 129, 128, 127, 284, 131, 126, 167, 130, 166, 132, 133, 285, 139, 134, 136, 137, 138, 140, 141, 142, 143, 144, 260, 261, 145, 148, 146, 147, 149, 150, 151, 152, 153, 275, 158, 276, 160, 161, 159, 162, 154, 155, 156, 157, 163, 164, 165, 170, 172, 168, 169, 171, 173, 177, 174, 178, 175, 179, 176, 182, 184, 185, 186, 181, 183, 187, 212, 191, 192, 213, 194, 190, 189, 188, 193, 195, 196, 203, 204, 198, 199, 200, 202, 201, 211, 218, 220,

[0, 15, 5, 19, 20, 21, 14, 12, 10, 22, 23, 24, 18, 25, 26, 27, 28, 29, 30, 31, 1, 9, 16, 55, 17, 3, 2, 41, 7, 8, 11, 4, 6, 13, 33, 39, 40, 45, 46, 47, 37, 48, 43, 49, 50, 51, 52, 53, 44, 54, 42, 35, 32, 36, 38, 34, 56, 57, 58, 59, 61, 60, 62, 64, 65, 66, 63, 68, 67, 69, 70, 77, 74, 83, 71, 73, 253, 254, 252, 82, 75, 76, 79, 81, 80, 78, 72, 136, 133, 117, 91, 134, 135, 86, 88, 90, 84, 85, 87, 89, 92, 93, 94, 95, 97, 101, 102, 103, 104, 96, 98, 99, 100, 128, 139, 140, 141, 142, 137, 138, 143, 121, 144, 110, 116, 156, 155, 182, 122, 118, 181, 105, 183, 184, 108, 157, 158, 106, 107, 109, 111, 112, 113, 114, 115, 119, 120, 124, 123, 125, 126, 127, 129, 130, 131, 132, 245, 281, 153, 150, 147, 149, 148, 152, 282, 154, 145, 146, 151, 274, 159, 217, 161, 167, 170, 166, 237, 173, 164, 169, 238, 160, 171, 215, 172, 216, 165, 168, 162, 163, 174, 180, 175, 176, 177, 178, 179, 190, 185, 214, 189, 186, 187, 188, 191, 193, 194, 192, 198, 200, 197, 201, 199, 352, 195, 196, 210, 205, 208, 202, 351, 203,

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 56, 57, 58, 53, 54, 45, 47, 48, 49, 50, 51, 52, 55, 94, 95, 84, 96, 97, 98, 60, 59, 93, 91, 99, 89, 82, 83, 90, 64, 92, 78, 79, 80, 76, 69, 85, 74, 77, 81, 71, 72, 86, 73, 87, 88, 63, 61, 62, 65, 66, 67, 68, 70, 75, 100, 101, 102, 103, 104, 193, 194, 195, 190, 121, 111, 152, 125, 173, 171, 113, 155, 129, 153, 120, 119, 174, 122, 107, 172, 116, 118, 212, 213, 123, 115, 214, 105, 151, 106, 134, 117, 154, 112, 108, 109, 110, 114, 124, 126, 127, 128, 130, 131, 132, 133, 353, 354, 355, 356, 135, 357, 137, 142, 136, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 150, 149, 161, 165, 175, 157, 182, 160, 189, 177, 156, 158, 162, 166, 170, 159, 163, 164, 167, 168, 169, 186, 191, 192, 178, 188, 176, 179, 180, 181, 183, 184, 185, 187, 293, 299, 246, 297, 294, 295, 245, 197, 251, 296, 276, 210, 298, 254, 275, 244, 279, 247,

...

[7, 10, 12, 13, 15, 21, 0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 30, 33, 34, 35, 37, 39, 36, 38, 40, 41, 42, 43, 44, 45, 46, 50, 53, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 72, 74, 69, 70, 71, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 96, 99, 93, 95, 97, 98, 100, 103, 104, 105, 101, 102, 107, 196, 115, 123, 384, 383, 297, 124, 127, 296, 195, 128, 109, 130, 197, 117, 112, 119, 120, 126, 110, 138, 214, 122, 116, 215, 216, 129, 131, 132, 108, 125, 106, 111, 113, 114, 118, 121, 144, 169, 168, 161, 162, 137, 163, 139, 164, 165, 166, 167, 143, 186, 187, 145, 135, 133, 134, 136, 140, 141, 142, 153, 182, 183, 184, 185, 357, 359, 363, 364, 175, 360, 365, 366, 155, 150, 173, 358, 158, 361, 362, 181, 381, 382, 380, 148, 151, 154, 149, 147, 152, 160, 146, 156, 157, 159, 172, 174, 170, 171, 177, 178, 179, 176, 180, 189, 192, 193, 188, 190, 191, 194, 201, 281, 204, 213,

C:\Users\kirstin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (39,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


[8, 68, 70, 71, 5, 18, 11, 21, 10, 28, 2, 27, 4, 22, 67, 24, 25, 20, 69, 12, 15, 0, 1, 3, 6, 7, 9, 13, 14, 16, 17, 19, 23, 26, 29, 31, 30, 32, 57, 47, 51, 33, 58, 39, 56, 49, 53, 54, 48, 38, 36, 44, 55, 52, 46, 35, 50, 37, 34, 40, 41, 42, 43, 45, 59, 60, 61, 62, 66, 63, 64, 65, 72, 73, 74, 75, 76, 77, 78, 80, 79, 81, 82, 83, 84, 85, 95, 91, 106, 94, 107, 93, 103, 90, 87, 88, 89, 92, 86, 145, 146, 104, 97, 96, 98, 99, 100, 101, 102, 105, 118, 119, 115, 111, 125, 117, 116, 109, 130, 131, 108, 110, 112, 113, 114, 120, 121, 123, 122, 128, 129, 124, 126, 127, 214, 213, 156, 198, 204, 209, 143, 210, 201, 211, 144, 139, 212, 203, 158, 154, 199, 200, 202, 227, 151, 149, 134, 174, 228, 173, 132, 148, 147, 142, 150, 135, 136, 141, 138, 133, 137, 140, 152, 153, 157, 155, 351, 316, 350, 314, 169, 313, 168, 315, 161, 167, 328, 329, 163, 330, 159, 160, 162, 165, 164, 166, 171, 172, 170, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 187, 195, 197, 190, 191, 192, 193, 194, 196,

[2, 3, 7, 10, 14, 15, 16, 19, 0, 1, 4, 5, 6, 8, 9, 11, 12, 13, 17, 18, 20, 22, 26, 23, 25, 27, 28, 29, 21, 30, 24, 40, 42, 36, 43, 39, 41, 34, 37, 35, 33, 31, 32, 38, 44, 45, 46, 49, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 60, 73, 74, 65, 118, 71, 64, 117, 67, 119, 120, 63, 72, 196, 68, 69, 70, 66, 78, 187, 111, 77, 80, 75, 76, 79, 81, 82, 84, 85, 83, 90, 112, 113, 114, 115, 116, 86, 87, 88, 89, 91, 92, 93, 97, 98, 94, 95, 96, 99, 101, 102, 100, 103, 104, 110, 168, 105, 109, 106, 129, 130, 171, 172, 169, 170, 107, 108, 131, 121, 124, 126, 132, 128, 122, 123, 125, 127, 143, 136, 272, 273, 255, 133, 271, 270, 135, 134, 137, 140, 248, 249, 250, 141, 138, 139, 145, 146, 142, 144, 147, 148, 149, 152, 153, 154, 150, 151, 155, 156, 167, 225, 158, 159, 224, 174, 161, 165, 223, 163, 162, 164, 182, 157, 160, 166, 173, 175, 176, 177, 181, 183, 184, 178, 179, 180, 185, 186, 188, 191, 195, 222, 189, 190, 192, 193, 194, 197, 198, 199, 201, 202, 200, 203, 204, 205, 206, 207, 220, 217,

[12, 10, 17, 18, 5, 8, 19, 13, 2, 15, 20, 24, 1, 16, 21, 22, 6, 23, 9, 0, 11, 25, 26, 14, 3, 4, 7, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 52, 51, 46, 43, 47, 42, 44, 45, 48, 49, 50, 53, 54, 57, 58, 59, 55, 56, 60, 61, 62, 63, 64, 66, 65, 72, 68, 99, 67, 69, 95, 97, 98, 96, 71, 86, 100, 134, 135, 103, 101, 102, 70, 73, 74, 83, 84, 81, 85, 87, 80, 77, 79, 82, 75, 76, 78, 88, 89, 90, 93, 91, 92, 94, 116, 189, 113, 188, 105, 190, 191, 109, 108, 110, 104, 143, 144, 107, 142, 106, 145, 146, 114, 115, 111, 112, 123, 159, 160, 165, 131, 163, 164, 122, 141, 120, 117, 118, 121, 119, 124, 130, 140, 125, 136, 127, 128, 132, 133, 126, 129, 139, 137, 138, 149, 261, 158, 148, 147, 150, 258, 152, 248, 156, 257, 155, 247, 153, 157, 260, 259, 151, 154, 161, 162, 166, 167, 168, 169, 170, 245, 246, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 244, 183, 182, 184, 185, 187, 186, 193, 237, 198, 197, 201, 195, 202, 192, 194, 196, 199, 200, 229, 213, 228, 204, 203, 205, 206, 207,

In [ ]:
import os

os.chdir('C:\\Users\\kirstin\\Desktop\\python')
import scipy.optimize as opt
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import argparse
import pickle
import time
import math
from skimage import io
io.use_plugin('matplotlib')
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
#import pyMods as pyM  
import gaussianfunctions_KM as gf
from scipy.interpolate import griddata
from scipy import stats
from pandas.core.window import Rolling
from skimage.measure import label, regionprops,regionprops_table
from skimage.color import label2rgb
from skimage.morphology import erosion, dilation, opening, closing
from skimage.morphology import disk

from skimage import data, util, measure
from skimage.segmentation import clear_border

#from skimage.external.tifffile import imsave
import peakutils
from PIL import Image
import tifffile
from scipy import ndimage
from skimage import img_as_ubyte
import shapely
from shapely.geometry import Polygon, Point
import statsmodels.api as sm
import numpy.ma as ma
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from matplotlib.patches import Ellipse
from skimage import img_as_uint
from skimage.draw import ellipse
from shapely.ops import nearest_points
from skimage import draw
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Circle
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

#generate output dataframe (one for the raw values of the med1 intenisty over time, and one for the average nuclear properties)
single_nuc_out = pd.DataFrame()
mean_out_df = pd.DataFrame()
single_mol_amp = 21
per_nuc_average = []
YAP = []
#define directory 
directory = "Z:\\Weiner_Lab\\kirstinmeyer\\YAP_condensate_paper\\YAP_hubs_vs_MED_NEW\\KY657_240629_YAP-SPT_Hubs\\Halo-YAP in Med_SNAP line\\"


#read in list of all result files in the current directory
os.chdir(directory)
csv_files = glob.glob("*result.csv")


for csv in csv_files:
    duration_per_nuc = []
    all_per_nuc = []
    
    #extract the roi from the csv file name
    roi = csv.split("_")[-3]
    roi_number = int(''.join(filter(str.isdigit, roi)))
    
    #read in 
    current_file = pd.read_csv(directory +csv)
    #current_file = raw_df.drop([0, 1,2])
    
    #read in file
    nuc_YAP_file = pd.read_csv(directory +csv.partition("_SPOTS")[0]+"_nuc_intensity.csv")
    
    #select the dataframe row that corresponds to the current nucleus roi (row 0 = roi 0, row 1 = roi 1 etc.)
    current_roi = nuc_YAP_file.iloc[roi_number]

    #get mean nuclear YAP intensity and area of the nucleus
    nuc_YAP = current_roi['Mean']   
    nuc_area = current_roi['Area']
    
    tracks = current_file['TRACK_ID']
    multi_frame_tracks = tracks.dropna(how='all')
    
    
    track_ID_int = [int(x) for x in multi_frame_tracks]
    unique_ids = unique(track_ID_int)
    print(unique_ids)
    
    for i in unique_ids:
        
            current_file["TRACK_ID"]= current_file["TRACK_ID"].astype(float)
            data = current_file.loc[current_file['TRACK_ID'] == float(i)]
            data4 = data.sort_values(by='new_frame',ascending=True, key=pd.to_numeric)
            
            #for trackmate, I allow a gap of two frames for calling it the same track. These are not listed in the original trackmate file.
            #I am therefore closing potential gaps for the time frames
            
            timeseries = data4['new_frame'].to_list()
            time_min = np.min(timeseries)
            time_max = np.max(timeseries)
            frame_list_full = [*range(time_min, time_max+1, 1)]       
            
            #making sure I am not copying over data from the previous loop, i am setting the list to empty
            YAP_timeseries = []
            YAP_timeseries = data4['YAP_amplitude'].to_list()
            for index, item in enumerate(YAP_timeseries):
                if item == "[0]":
                    YAP_timeseries[index] = "0"
            YAP_timeseries_float = [float(x) for x in YAP_timeseries]
            
            #only consider trace a"YAP hub" if intenisty at any point is >=5 molecules
            if any(float(num) >= 5*single_mol_amp  for num in YAP_timeseries_float): 
                YAP_min = np.nan
                YAP_max = np.nan
                YAP_mean = np.nan
                YAP_median = np.nan

                duration = np.nan
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [YAP_min] + [YAP_max] + [YAP_mean] + [YAP_median] + [duration]
                mean_out_df =mean_out_df.append(pd.Series(out),ignore_index=True)
                #mean_out_df[str(csv) + str(i)] = pd.Series(out)
                
                
                
            else:
                YAP_min = np.min(YAP_timeseries_float)
                YAP_max = np.max(YAP_timeseries_float)
                YAP_mean = np.mean(YAP_timeseries_float)
                YAP_median = np.median(YAP_timeseries_float)
                duration = len(frame_list_full)
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [YAP_min] + [YAP_max] + [YAP_mean] + [YAP_median] + [duration]
                mean_out_df = mean_out_df.append(pd.Series(out),ignore_index=True)           
                
                all_per_nuc = all_per_nuc + [YAP_mean]
                duration_per_nuc = duration_per_nuc + [duration]
                

    mean_YAP_per_nuc = np.mean(all_per_nuc)
    median_YAP_per_nuc = np.median(all_per_nuc)
    num_hubs_per_nuc = len(all_per_nuc)
    mean_duration = np.mean(duration_per_nuc)
    
    per_nuc_out =  [nuc_YAP]+ [nuc_area]+ [mean_YAP_per_nuc] + [median_YAP_per_nuc] + [num_hubs_per_nuc] +[mean_duration]
    single_nuc_out =single_nuc_out.append(pd.Series(per_nuc_out),ignore_index=True)
    
single_nuc_out.to_csv(directory +"KY657_YAP_hub_features_per_nucleus_amplitude_bigger_5_molecule.csv")           
#mean_out_df.to_csv(directory +"YAP_hub_features.csv")    


In [ ]:
import os

os.chdir('C:\\Users\\kirstin\\Desktop\\python')
import scipy.optimize as opt
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import argparse
import pickle
import time
import math
from skimage import io
io.use_plugin('matplotlib')
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
#import pyMods as pyM  
import gaussianfunctions_KM as gf
from scipy.interpolate import griddata
from scipy import stats
from pandas.core.window import Rolling
from skimage.measure import label, regionprops,regionprops_table
from skimage.color import label2rgb
from skimage.morphology import erosion, dilation, opening, closing
from skimage.morphology import disk

from skimage import data, util, measure
from skimage.segmentation import clear_border

#from skimage.external.tifffile import imsave
import peakutils
from PIL import Image
import tifffile
from scipy import ndimage
from skimage import img_as_ubyte
import shapely
from shapely.geometry import Polygon, Point
import statsmodels.api as sm
import numpy.ma as ma
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from matplotlib.patches import Ellipse
from skimage import img_as_uint
from skimage.draw import ellipse
from shapely.ops import nearest_points
from skimage import draw
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Circle
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

#generate output dataframe (one for the raw values of the med1 intenisty over time, and one for the average nuclear properties)
single_nuc_out = pd.DataFrame()
mean_out_df = pd.DataFrame()
single_mol_amp = 212
per_nuc_average = []
YAP = []

directory = "Z:\\Weiner_Lab\\kirstinmeyer\\scope\\SINGLE-MOLECULE_IMAGING\\KY612_240403_Med_YAP_hubs_with_charge\\Halo-YAP(WT)\\"

#read in list of all result files in the current directory
os.chdir(directory)
csv_files = glob.glob("*result.csv")


for csv in csv_files:
    print(csv)
    duration_per_nuc = []
    all_per_nuc = []
    all_per_nuc_amp =[]
    
    #extract the roi from the csv file name
    roi = csv.split("_")[-3]
    roi_number = int(''.join(filter(str.isdigit, roi)))
    
    #read in 
    current_file = pd.read_csv(directory +csv)
    #current_file = raw_df.drop([0, 1,2])
    
    #read in file
    nuc_YAP_file = pd.read_csv(directory +csv.partition("_SPOTS")[0]+"_nuc_intensity.csv")
    
    #select the dataframe row that corresponds to the current nucleus roi (row 0 = roi 0, row 1 = roi 1 etc.)
    current_roi = nuc_YAP_file.iloc[roi_number]

    #get mean nuclear YAP intensity and area of the nucleus
    nuc_YAP = current_roi['Mean']   
    nuc_area = current_roi['Area']
    
    tracks = current_file['TRACK_ID']
    multi_frame_tracks = tracks.dropna(how='all')
    
    
    track_ID_int = [int(x) for x in multi_frame_tracks]
    unique_ids = unique(track_ID_int)
    print(unique_ids)
    
    for i in unique_ids:
        
            current_file["TRACK_ID"]= current_file["TRACK_ID"].astype(float)
            data = current_file.loc[current_file['TRACK_ID'] == float(i)]
            data4 = data.sort_values(by='new_frame',ascending=True, key=pd.to_numeric)
            
            #for trackmate, I allow a gap of two frames for calling it the same track. These are not listed in the original trackmate file.
            #I am therefore closing potential gaps for the time frames
            
            timeseries = data4['new_frame'].to_list()
            time_min = np.min(timeseries)
            time_max = np.max(timeseries)
            frame_list_full = [*range(time_min, time_max+1, 1)]       
            
            #making sure I am not copying over data from the previous loop, i am setting the list to empty
            Med_timeseries = []
            Med_timeseries = data4['YAP_integral_int'].to_list()
            Med_timeseries_amplitude = data4['YAP_amplitude'].to_list()
            for index, item in enumerate(Med_timeseries):
                if item == "[0]":
                    Med_timeseries[index] = "0"
            
            Med_timeseries_float = [float(x) for x in Med_timeseries]
            
            for index, item in enumerate(Med_timeseries_amplitude):
                if item == "[0]":
                    Med_timeseries_amplitude[index] = "0"
            
            Med_timeseries_amplitude_float = [float(x) for x in Med_timeseries_amplitude]
            
            #only consider trace a"YAP hub" if intenisty at any point is >=5 molecules
            if any(float(num) >= 3*single_mol_amp  for num in Med_timeseries_float): 
                Med_min = np.min(Med_timeseries_float)
                Med_max = np.max(Med_timeseries_float)
                Med_mean = np.mean(Med_timeseries_float)
                Med_median = np.median(Med_timeseries_float)
                Med_mean_amplitude = np.mean(Med_timeseries_amplitude_float)
                
                
                duration = len(frame_list_full)
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [Med_min] + [Med_max] + [Med_mean] + [Med_median] + [duration] + [Med_mean_amplitude]
                mean_out_df = mean_out_df.append(pd.Series(out),ignore_index=True)           
                
                all_per_nuc = all_per_nuc + [Med_mean]
                all_per_nuc_amp = all_per_nuc_amp + [Med_mean_amplitude]
                
                duration_per_nuc = duration_per_nuc + [duration]
            else:
                
                Med_min = np.nan
                Med_max = np.nan
                Med_mean = np.nan
                Med_median = np.nan
                Med_mean_amplitude = np.nan

                duration = np.nan
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [Med_min] + [Med_max] + [Med_mean] + [Med_median] + [duration]
                mean_out_df =mean_out_df.append(pd.Series(out),ignore_index=True)
                #mean_out_df[str(csv) + str(i)] = pd.Series(out)
    mean_Med_per_nuc = np.mean(all_per_nuc)
    mean_Med_amplitude_based_per_nuc = np.mean(all_per_nuc_amp)
    median_Med_per_nuc = np.median(all_per_nuc)
    num_hubs_per_nuc = len(all_per_nuc)
    mean_duration = np.mean(duration_per_nuc)
    number_all_tracks = len(unique_ids)
    
    per_nuc_out =  [nuc_YAP]+ [nuc_area]+ [mean_Med_per_nuc] + [median_Med_per_nuc] + [num_hubs_per_nuc] +[mean_duration] + [number_all_tracks] +[mean_Med_amplitude_based_per_nuc]
    single_nuc_out =single_nuc_out.append(pd.Series(per_nuc_out),ignore_index=True)
    
single_nuc_out.to_csv(directory +"KY612_Halo-YAP_YAP_hub_features_per_nucleus_integral_int_samller 2molecules.csv")           
mean_out_df.to_csv(directory +"KY612_Halo-YAP_YAP_hub_features_all_tracks_integral_int_smaller_2molecules.csv")    


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
import os

os.chdir('C:\\Users\\kirstin\\Desktop\\python')
import scipy.optimize as opt
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import argparse
import pickle
import time
import math
from skimage import io
io.use_plugin('matplotlib')
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
#import pyMods as pyM  
import gaussianfunctions_KM as gf
from scipy.interpolate import griddata
from scipy import stats
from pandas.core.window import Rolling
from skimage.measure import label, regionprops,regionprops_table
from skimage.color import label2rgb
from skimage.morphology import erosion, dilation, opening, closing
from skimage.morphology import disk

from skimage import data, util, measure
from skimage.segmentation import clear_border

#from skimage.external.tifffile import imsave
import peakutils
from PIL import Image
import tifffile
from scipy import ndimage
from skimage import img_as_ubyte
import shapely
from shapely.geometry import Polygon, Point
import statsmodels.api as sm
import numpy.ma as ma
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from matplotlib.patches import Ellipse
from skimage import img_as_uint
from skimage.draw import ellipse
from shapely.ops import nearest_points
from skimage import draw
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Circle
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

#generate output dataframe (one for the raw values of the med1 intenisty over time, and one for the average nuclear properties)
single_nuc_out = pd.DataFrame()
mean_out_df = pd.DataFrame()
single_mol_amp = 212
per_nuc_average = []
YAP = []

directory = "Z:\\Weiner_Lab\\kirstinmeyer\\scope\\SINGLE-MOLECULE_IMAGING\\KY613_240407_Med_YAP_hubs_charge\\KY613_240407_Med_YAP_hubs_charge\\Halo-YAP(WT)\\"

#read in list of all result files in the current directory
os.chdir(directory)
csv_files = glob.glob("*result.csv")


for csv in csv_files:
    print(csv)
    duration_per_nuc = []
    all_per_nuc = []
    all_per_nuc_amp =[]
    
    #extract the roi from the csv file name
    roi = csv.split("_")[-3]
    roi_number = int(''.join(filter(str.isdigit, roi)))
    
    #read in 
    current_file = pd.read_csv(directory +csv)
    #current_file = raw_df.drop([0, 1,2])
    
    #read in file
    nuc_YAP_file = pd.read_csv(directory +csv.partition("_SPOTS")[0]+"_nuc_intensity.csv")
    
    #select the dataframe row that corresponds to the current nucleus roi (row 0 = roi 0, row 1 = roi 1 etc.)
    current_roi = nuc_YAP_file.iloc[roi_number]

    #get mean nuclear YAP intensity and area of the nucleus
    nuc_YAP = current_roi['Mean']   
    nuc_area = current_roi['Area']
    
    tracks = current_file['TRACK_ID']
    multi_frame_tracks = tracks.dropna(how='all')
    
    
    track_ID_int = [int(x) for x in multi_frame_tracks]
    unique_ids = unique(track_ID_int)
    print(unique_ids)
    
    for i in unique_ids:
        
            current_file["TRACK_ID"]= current_file["TRACK_ID"].astype(float)
            data = current_file.loc[current_file['TRACK_ID'] == float(i)]
            data4 = data.sort_values(by='new_frame',ascending=True, key=pd.to_numeric)
            
            #for trackmate, I allow a gap of two frames for calling it the same track. These are not listed in the original trackmate file.
            #I am therefore closing potential gaps for the time frames
            
            timeseries = data4['new_frame'].to_list()
            time_min = np.min(timeseries)
            time_max = np.max(timeseries)
            frame_list_full = [*range(time_min, time_max+1, 1)]       
            
            #making sure I am not copying over data from the previous loop, i am setting the list to empty
            Med_timeseries = []
            Med_timeseries = data4['YAP_integral_int'].to_list()
            Med_timeseries_amplitude = data4['YAP_amplitude'].to_list()
            for index, item in enumerate(Med_timeseries):
                if item == "[0]":
                    Med_timeseries[index] = "0"
            
            Med_timeseries_float = [float(x) for x in Med_timeseries]
            
            for index, item in enumerate(Med_timeseries_amplitude):
                if item == "[0]":
                    Med_timeseries_amplitude[index] = "0"
            
            Med_timeseries_amplitude_float = [float(x) for x in Med_timeseries_amplitude]
            
            #only consider trace a"YAP hub" if intenisty at any point is >=5 molecules
            if any(float(num) >= 2*single_mol_amp  for num in Med_timeseries_float): 
                Med_min = np.nan
                Med_max = np.nan
                Med_mean = np.nan
                Med_median = np.nan
                Med_mean_amplitude = np.nan

                duration = np.nan
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [Med_min] + [Med_max] + [Med_mean] + [Med_median] + [duration]
                mean_out_df =mean_out_df.append(pd.Series(out),ignore_index=True)
                #mean_out_df[str(csv) + str(i)] = pd.Series(out)

            else:
                Med_min = np.min(Med_timeseries_float)
                Med_max = np.max(Med_timeseries_float)
                Med_mean = np.mean(Med_timeseries_float)
                Med_median = np.median(Med_timeseries_float)
                Med_mean_amplitude = np.mean(Med_timeseries_amplitude_float)
                
                
                duration = len(frame_list_full)
                out =[str(csv) + str(i)]+ [nuc_YAP] + [nuc_area] + [Med_min] + [Med_max] + [Med_mean] + [Med_median] + [duration] + [Med_mean_amplitude]
                mean_out_df = mean_out_df.append(pd.Series(out),ignore_index=True)           
                
                all_per_nuc = all_per_nuc + [Med_mean]
                all_per_nuc_amp = all_per_nuc_amp + [Med_mean_amplitude]
                
                duration_per_nuc = duration_per_nuc + [duration]
                
                
    mean_Med_per_nuc = np.mean(all_per_nuc)
    mean_Med_amplitude_based_per_nuc = np.mean(all_per_nuc_amp)
    median_Med_per_nuc = np.median(all_per_nuc)
    num_hubs_per_nuc = len(all_per_nuc)
    mean_duration = np.mean(duration_per_nuc)
    number_all_tracks = len(unique_ids)
    
    per_nuc_out =  [nuc_YAP]+ [nuc_area]+ [mean_Med_per_nuc] + [median_Med_per_nuc] + [num_hubs_per_nuc] +[mean_duration] + [number_all_tracks] +[mean_Med_amplitude_based_per_nuc]
    single_nuc_out =single_nuc_out.append(pd.Series(per_nuc_out),ignore_index=True)
    
single_nuc_out.to_csv(directory +"KY613_Halo-YAP_YAP_hub_features_per_nucleus_integral_int_samller 2molecules.csv")           
mean_out_df.to_csv(directory +"KY613_Halo-YAP_YAP_hub_features_all_tracks_integral_int_smaller_2molecules.csv")    
